In [1]:
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
import os
import pandas as pd

os.environ['GOOGLE_API_KEY'] =  ''

In [2]:
# prepare pdf
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
pdf_loader = PyPDFLoader("impromptu-rh.pdf")

text_splitter = RecursiveCharacterTextSplitter(separators = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len
)

pdf_data = pdf_loader.load_and_split(text_splitter=text_splitter)

In [4]:
from langchain.vectorstores import Chroma

embeddings = GooglePalmEmbeddings()

vectordb = Chroma.from_documents(
    pdf_data, 
    embedding=embeddings, 
    persist_directory="."
)

vectordb.persist()

In [6]:
# initialize chat
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = ConversationalRetrievalChain.from_llm(
    GooglePalm(), 
    retriever=vectordb.as_retriever(), 
    memory=memory,
)
     

In [8]:
# chat
chat_history = []

while True:
    user_input = input("> ")
    ai_response = conversation({
        "question": user_input,
        "chat_history": chat_history
    })
    print(ai_response['answer'])

> what is open ai ?
a research organization founded in 2015 with a mission to give millions of people direct, hands-on access to powerful new AI tools.
> what is their objective ?
OpenAI's objective is to ensure that its LLMs are used for good, and to reduce their capacity to generate harmful, unethical, and unsafe outputs.
> who is the founder ?
Sam Altman


KeyboardInterrupt: Interrupted by user